# Install MQTT Library

With Conda (if you are using Anaconda)

In [ ]:
import sys
!conda install -y -p {sys.prefix} -c conda-forge paho-mqtt

With Pip

In [ ]:
!pip install paho-mqtt

Verify that we have the library properly installed

The following code block should not generate an error

In [ ]:
import paho.mqtt.client as mqtt

# Parameters

In [ ]:
# Replace with your matric ID
YOUR_MATRIC_ID = 'A1234567Z'
# Note that the sensors publish a reading every few seconds
NUM_SAMPLES = 1

# The following does not need to be changed
MQTT_HOST = '10.246.112.87'
MQTT_SUBPATH = 'ee4211/'
MQTT_LIGHT_SENSOR = 'light_sensor_0/brightness_lux'
MQTT_TEMP_SENSOR = 'temp_sensor_0/temperature_degC'

# Data Collection

Define callback function used to collect published data

In [ ]:
import json

def parse_json(msg):
    err = False
    try:
        json_object = json.loads(msg)
    except:
        err = True
    else:
        if 'v' not in json_object or 'ts' not in json_object:
            err = True
    if err:
        print('Malformed reading received:', end=' ')
        print(msg)
        return None
    return json_object

light_readings = []
def collect_received_light(c, u, msg):
    msg = parse_json(msg.payload)
    if not msg: return

    # Two columns, 'timestamp' and 'brightness_lux'
    reading = {}
    # The timestamp is contained in msg['ts']
    reading['timestamp'] = msg['ts']
    # The reading is contained in msg['v']
    reading['brightness_lux'] = msg['v']
    # Cast the reading into an integer
    reading['brightness_lux'] = int(reading['brightness_lux'])
    print('Got light reading:', end=' ')
    print(reading)
    # Append it to the 'light_readings' list
    light_readings.append(reading)

temp_readings = []
def collect_received_temp(c, u, msg):
    msg = parse_json(msg.payload)
    if not msg: return

    # Two columns, 'timestamp' and 'temperature_degC'
    reading = {}
    # The timestamp is contained in msg['ts']
    reading['timestamp'] = msg['ts']
    # The reading is contained in msg['v']
    reading['temperature_degC'] = msg['v']
    # Cast the reading into a float
    reading['temperature_degC'] = float(reading['temperature_degC'])
    print('Got temp reading:', end=' ')
    print(reading)
    # Append it to the 'temp_readings' list
    temp_readings.append(reading)

def collect_received(c, u, msg):
    if MQTT_LIGHT_SENSOR in msg.topic:
        collect_received_light(c, u, msg)
    elif MQTT_TEMP_SENSOR in msg.topic:
        collect_received_temp(c, u, msg)

Generate anonymised ID for connection to MQTT server

In [ ]:
import hashlib
MQTT_ID = YOUR_MATRIC_ID.lower().strip()
MQTT_ID = hashlib.sha256(MQTT_ID.encode()).hexdigest()[-16:]

Connect to MQTT server

In [ ]:
import paho.mqtt.client as mqtt
client = mqtt.Client()
# Attach the on_message callback to the collect_received() function
client.on_message = collect_received
client.username_pw_set(username = MQTT_ID, password = MQTT_ID)
client.connect(MQTT_HOST)
client.loop_start()

Subscribe to sensor topics

In [ ]:
# Clear the list of readings first
light_readings = []
temp_readings = []

MQTT_TOPIC = MQTT_SUBPATH + MQTT_ID + '/' + MQTT_LIGHT_SENSOR
client.subscribe(MQTT_TOPIC)
print('Subscribed to topic:', end=' ')
print(MQTT_TOPIC)

MQTT_TOPIC = MQTT_SUBPATH + MQTT_ID + '/' + MQTT_TEMP_SENSOR
client.subscribe(MQTT_TOPIC)
print('Subscribed to topic:', end=' ')
print(MQTT_TOPIC)

Wait for the required number of readings to accumulate and disconnect.

If you do not see any readings being collected within a few seconds, something is wrong. Check that you have your matric id filled in correctly.

In [ ]:
from time import sleep
while len(light_readings) < NUM_SAMPLES or len(temp_readings) < NUM_SAMPLES:
    sleep(1.0)
client.disconnect()
client.loop_stop()

print('Number of light readings obtained:', end=' ')
print(len(light_readings))

print('Number of temp readings obtained:', end=' ')
print(len(temp_readings))

Dump the readings to a JSON file for offline processing

In [ ]:
import json
import time
curr_time = int(time.time())
with open(f'readings_{curr_time}_light.json', 'w') as f:
    json.dump(light_readings, f)
with open(f'readings_{curr_time}_temp.json', 'w') as f:
    json.dump(temp_readings, f)

# Data Processing

Write your data processing functions to answer the questions in the worksheet.

Your normal data visualisation and processing libraries such as seaborn, pandas and numpy are available.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

import json
from datetime import datetime

def parse_timestamp(ts):
    try:
        # Requires Python 3.7 and above
        ts = datetime.fromisoformat(ts)
    except:
        # Python version does not have fromisoformat(), we continue
        # to represent the timestamp as a string.
        # It is then up to you to properly parse the timestamp in
        # your processing
        pass
    return ts

# Sample that just imports from json and prints all the readings
run_time = curr_time
with open(f'readings_{run_time}_light.json', 'r') as f:
    light_readings = json.load(f)
with open(f'readings_{run_time}_temp.json', 'r') as f:
    temp_readings = json.load(f)

for light_reading in light_readings:
    light_reading['timestamp'] = parse_timestamp(light_reading['timestamp'])
    print('Timestamp:', end=' ')
    print(light_reading['timestamp'], end=' ')
    print('Brightness (lux):', end=' ')
    print(light_reading['brightness_lux'])

for temp_reading in temp_readings:
    temp_reading['timestamp'] = parse_timestamp(temp_reading['timestamp'])
    print('Timestamp:', end=' ')
    print(temp_reading['timestamp'], end=' ')
    print('Temperature (degC):', end=' ')
    print(temp_reading['temperature_degC'])